## Chicago Crime ETL/Feature Engineering

Engineering features at the beat/year level for 2016-2019

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
crime_df_path = '../data/Crimes_-_2001_to_Present.csv'
crime_df = pd.read_csv(crime_df_path)

In [3]:
crime_df.shape

(7306290, 22)

In [4]:
crime_df['Date'] = pd.to_datetime(crime_df['Date'], format = '%m/%d/%Y %I:%M:%S %p')
crime_df['day_date'] = crime_df['Date'].dt.date

#### Subset the 2016-2019 data

In [5]:
crime_df_16_19 = crime_df[crime_df.day_date > datetime.date(2015, 12, 31)]
crime_df_16_19 = crime_df_16_19[crime_df_16_19.day_date < datetime.date(2020, 1, 1)]
min_date, max_date = crime_df_16_19.day_date.min(), crime_df_16_19.day_date.max()
print(f'Min date: {min_date} and Max date: {max_date}')

Min date: 2016-01-01 and Max date: 2019-12-31


#### Features Engineering

In [6]:
#rename for ease
cdf = crime_df_16_19

In [7]:
print(cdf.shape)
print(cdf.columns)
cdf.dtypes

(1067294, 23)
Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'day_date'],
      dtype='object')


ID                               int64
Case Number                     object
Date                    datetime64[ns]
Block                           object
IUCR                            object
Primary Type                    object
Description                     object
Location Description            object
Arrest                            bool
Domestic                          bool
Beat                             int64
District                       float64
Ward                           float64
Community Area                 float64
FBI Code                        object
X Coordinate                   float64
Y Coordinate                   float64
Year                             int64
Updated On                      object
Latitude                       float64
Longitude                      float64
Location                        object
day_date                        object
dtype: object

#### Group the data by beat and year:

In [8]:
cdf_beats = pd.DataFrame(cdf.groupby(['Year', 'Beat'])['ID'].count())
cdf_beats.rename(columns={'ID': 'report_cnt'}, inplace=True)
print(cdf_beats.shape)
cdf_beats.head(5)

(1096, 1)


report_cnt
Year Beat            
2016 111         1843
     112         1849
     113         1085
     114         1049
     121          810

#### Create counts for each Primary Type at the beat/year level

In [9]:
print(cdf['Primary Type'].value_counts())

THEFT                                253701
BATTERY                              198846
CRIMINAL DAMAGE                      114565
ASSAULT                               79067
DECEPTIVE PRACTICE                    76996
OTHER OFFENSE                         68571
NARCOTICS                             53644
BURGLARY                              48669
MOTOR VEHICLE THEFT                   41622
ROBBERY                               41511
CRIMINAL TRESPASS                     26846
WEAPONS VIOLATION                     19931
OFFENSE INVOLVING CHILDREN             9369
PUBLIC PEACE VIOLATION                 5997
CRIM SEXUAL ASSAULT                    5441
INTERFERENCE WITH PUBLIC OFFICER       4875
SEX OFFENSE                            4539
PROSTITUTION                           2933
HOMICIDE                               2566
ARSON                                  1708
CRIMINAL SEXUAL ASSAULT                1097
LIQUOR LAW VIOLATION                    918
STALKING                        

In [10]:
#Adding primary type counts to each beat-year
cdf_beats.reset_index(inplace=True)
by_primary = pd.DataFrame(cdf.groupby(['Year', 'Beat', 'Primary Type'])['ID'].count())
by_primary.reset_index(inplace=True)
primaries = list(cdf['Primary Type'].value_counts().index)
for primary in primaries:
    filtered = by_primary[by_primary['Primary Type'] == primary]
    filtered.drop('Primary Type', axis =1, inplace=True)
    filtered.rename(columns={'ID': primary}, inplace=True)
    cdf_beats = cdf_beats.merge(filtered, how='left', on=['Year', 'Beat'])


cdf_beats

/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Year,Beat,report_cnt,THEFT,BATTERY,CRIMINAL DAMAGE,ASSAULT,DECEPTIVE PRACTICE,OTHER OFFENSE,NARCOTICS,...,GAMBLING,INTIMIDATION,CONCEALED CARRY LICENSE VIOLATION,OBSCENITY,NON-CRIMINAL,HUMAN TRAFFICKING,PUBLIC INDECENCY,OTHER NARCOTIC VIOLATION,NON-CRIMINAL (SUBJECT SPECIFIED),NON - CRIMINAL
0,2016,111,1843,1143,116,57.0,84,222.0,37,17.0,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,112,1849,1115,123,41.0,55,318.0,44,5.0,...,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,2016,113,1085,482,151,58.0,46,149.0,21,24.0,...,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN
3,2016,114,1049,558,93,53.0,36,132.0,32,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,121,810,349,63,63.0,27,118.0,46,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2019,2531,690,108,158,71.0,62,67.0,58,43.0,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1092,2019,2532,1026,148,237,104.0,78,82.0,92,74.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1093,2019,2533,1604,503,241,137.0,104,120.0,65,96.0,...,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1094,2019,2534,1192,189,270,149.0,118,44.0,72,68.0,...,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
cdf_beats.isna().sum()

Year                                    0
Beat                                    0
report_cnt                              0
THEFT                                   0
BATTERY                                 0
CRIMINAL DAMAGE                         1
ASSAULT                                 0
DECEPTIVE PRACTICE                      1
OTHER OFFENSE                           0
NARCOTICS                               7
BURGLARY                               19
MOTOR VEHICLE THEFT                    11
ROBBERY                                14
CRIMINAL TRESPASS                       0
WEAPONS VIOLATION                      43
OFFENSE INVOLVING CHILDREN             45
PUBLIC PEACE VIOLATION                 53
CRIM SEXUAL ASSAULT                    49
INTERFERENCE WITH PUBLIC OFFICER      200
SEX OFFENSE                            47
PROSTITUTION                          862
HOMICIDE                              330
ARSON                                 371
CRIMINAL SEXUAL ASSAULT           

Some of these uncommon primary types could be dummies

#### Create rates for each Primary Type

In [12]:
for primary in primaries:
    col_name = primary + "_rate"
    cdf_beats[col_name] = cdf_beats[primary] / cdf_beats['report_cnt']
    
cdf_beats

,Year,Beat,report_cnt,THEFT,BATTERY,CRIMINAL DAMAGE,ASSAULT,DECEPTIVE PRACTICE,OTHER OFFENSE,NARCOTICS,...,GAMBLING_rate,INTIMIDATION_rate,CONCEALED CARRY LICENSE VIOLATION_rate,OBSCENITY_rate,NON-CRIMINAL_rate,HUMAN TRAFFICKING_rate,PUBLIC INDECENCY_rate,OTHER NARCOTIC VIOLATION_rate,NON-CRIMINAL (SUBJECT SPECIFIED)_rate,NON - CRIMINAL_rate
0,2016,111,1843,1143,116,57.0,84,222.0,37,17.0,...,NaN,NaN,NaN,0.000543,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,112,1849,1115,123,41.0,55,318.0,44,5.0,...,NaN,NaN,NaN,0.000541,0.000541,NaN,NaN,NaN,NaN,NaN
2,2016,113,1085,482,151,58.0,46,149.0,21,24.0,...,NaN,NaN,NaN,NaN,0.001843,NaN,0.001843,NaN,NaN,NaN
3,2016,114,1049,558,93,53.0,36,132.0,32,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,121,810,349,63,63.0,27,118.0,46,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2019,2531,690,108,158,71.0,62,67.0,58,43.0,...,NaN,0.001449,0.001449,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1092,2019,2532,1026,148,237,104.0,78,82.0,92,74.0,...,NaN,0.000975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1093,2019,2533,1604,503,241,137.0,104,120.0,65,96.0,...,NaN,0.000623,0.001870,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1094,2019,2534,1192,189,270,149.0,118,44.0,72,68.0,...,NaN,0.001678,0.001678,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Add arrest rate and domestic rate for each beat/year

In [13]:
arrest_cnt = pd.DataFrame(cdf[cdf['Arrest'] == True].groupby(['Year', 'Beat'])['Arrest'].count())
domestic_cnt = pd.DataFrame(cdf[cdf['Domestic'] == True].groupby(['Year', 'Beat'])['Domestic'].count())

In [14]:
for df in [arrest_cnt, domestic_cnt]:
    df.reset_index(inplace=True)
    cdf_beats = cdf_beats.merge(df, how='left', on=['Year', 'Beat'])

In [15]:
cdf_beats['Arrest_rate'] = cdf_beats['Arrest'] / cdf_beats['report_cnt']
cdf_beats['Domestic_rate'] = cdf_beats['Domestic'] / cdf_beats['report_cnt']
cdf_beats

,Year,Beat,report_cnt,THEFT,BATTERY,CRIMINAL DAMAGE,ASSAULT,DECEPTIVE PRACTICE,OTHER OFFENSE,NARCOTICS,...,NON-CRIMINAL_rate,HUMAN TRAFFICKING_rate,PUBLIC INDECENCY_rate,OTHER NARCOTIC VIOLATION_rate,NON-CRIMINAL (SUBJECT SPECIFIED)_rate,NON - CRIMINAL_rate,Arrest,Domestic,Arrest_rate,Domestic_rate
0,2016,111,1843,1143,116,57.0,84,222.0,37,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,561,47.0,0.304395,0.025502
1,2016,112,1849,1115,123,41.0,55,318.0,44,5.0,...,0.000541,NaN,NaN,NaN,NaN,NaN,377,29.0,0.203894,0.015684
2,2016,113,1085,482,151,58.0,46,149.0,21,24.0,...,0.001843,NaN,0.001843,NaN,NaN,NaN,199,33.0,0.183410,0.030415
3,2016,114,1049,558,93,53.0,36,132.0,32,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,124,44.0,0.118208,0.041945
4,2016,121,810,349,63,63.0,27,118.0,46,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,129,30.0,0.159259,0.037037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2019,2531,690,108,158,71.0,62,67.0,58,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,150,140.0,0.217391,0.202899
1092,2019,2532,1026,148,237,104.0,78,82.0,92,74.0,...,NaN,NaN,NaN,NaN,NaN,NaN,219,273.0,0.213450,0.266082
1093,2019,2533,1604,503,241,137.0,104,120.0,65,96.0,...,NaN,NaN,NaN,NaN,NaN,NaN,464,193.0,0.289277,0.120324
1094,2019,2534,1192,189,270,149.0,118,44.0,72,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,250,226.0,0.209732,0.189597


In [16]:
cdf_beats.to_csv('../data/features/crime.csv')